# ADSP_analysis_update

This notenook is for the  analysis on ADSP WES 

Written by Bale, 2023

***Pheno data***: /mnt/mfs/statgen/alzheimers-family/ADSP_pheno_updated_20221129/

***Genetic data***:  /mnt/mfs/statgen/alzheimers-family/ADSP/merge_adsp
    
We used GMMAT software included in [our workflow](https://github.com/cumc/bioworkflows/blob/master/GWAS/LMM.ipynb) for the analyses. The docker image that contains all the necessary software installations to run this notebook and workflow can be retrieved by the following command: docker pull statisticalgenetics/lmm:3.0

# Common Variants

## QC genetic data

In [8]:
#Generating genoFile without LD pruning
ml Singularity
for i in African European Hispanic; do
# unrelated individuals
sos run ~/project2022/notebook/AD/xqtl-pipeline/pipeline/GWAS_QC.ipynb qc_no_prune \
    --cwd /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno \
    --genoFile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno/merge_adsp.$i.filtered.bed \
    --keep_samples /mnt/mfs/statgen/alzheimers-family/ADSP_2023/PCA/king/merge_adsp.$i.filtered.unrelated_id \
    --remove_samples /mnt/mfs/statgen/alzheimers-family/ADSP_2023/PCA_QC/$i.pca.projected.outliers \
    --maf_filter 0.01 \
    --geno_filter 0.01 \
    --mind_filter 0.1 \
    --hwe_filter 5e-08 \
    --name unrelated_common \
    --container /mnt/mfs/statgen/containers/lmm.sif
done
# related individuals same set of variants
for i in African European Hispanic; do
sos run ~/project2022/notebook/AD/xqtl-pipeline/pipeline/GWAS_QC.ipynb qc_no_prune \
    --cwd /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno \
    --genoFile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno/merge_adsp.$i.filtered.bed \
    --keep_samples /mnt/mfs/statgen/alzheimers-family/ADSP_2023/PCA/king/merge_adsp.$i.filtered.related_id\
    --remove_samples /mnt/mfs/statgen/alzheimers-family/ADSP_2023/PCA_QC/$i.pca.projected.outliers \
    --keep_variants /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno/merge_adsp.$i.filtered.unrelated_common.filtered.bim \
    --maf_filter 0 \
    --geno_filter 1 \
    --mind_filter 1 \
    --hwe_filter 5e-08 \
    --name related \
    --container /mnt/mfs/statgen/containers/lmm.sif 
done

INFO: Running qc_no_prune: Filter SNPs and select individuals
INFO: qc_no_prune (index=0) is ignored due to saved signature
INFO: qc_no_prune output:   /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno/merge_adsp.African.filtered.unrelated_common.filtered.bed
INFO: Workflow qc_no_prune (ID=w1ea7ee50c07bd868) is ignored with 1 ignored step.
INFO: Running qc_no_prune: Filter SNPs and select individuals
INFO: qc_no_prune (index=0) is ignored due to saved signature
INFO: qc_no_prune output:   /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno/merge_adsp.European.filtered.unrelated_common.filtered.bed
INFO: Workflow qc_no_prune (ID=w60acd67624b6bdc5) is ignored with 1 ignored step.
INFO: Running qc_no_prune: Filter SNPs and select individuals
INFO: qc_no_prune (index=0) is ignored due to saved signature
INFO: qc_no_prune output:   /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno/merge_adsp.Hispanic.filtered.unrelated_common.filtered.bed
INFO: Workflow qc_no_prune (ID=wc7ec1182c56c2517) is i

In [ ]:
ml Singularity
for i in African European Hispanic; do
sos run ~/project2022/notebook/AD/xqtl-pipeline/pipeline/GWAS_QC.ipynb qc:1 \
  --cwd /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno/qced \
  --genoFile /mnt/mfs/statgen/alzheimers-family/ADSP/merge_adsp.bed \
  --maf_filter 0.01 \
  --geno_filter 0.01 \
  --mind_filter 0.1 \
  --keep_samples /mnt/mfs/statgen/alzheimers-family/ADSP_pheno_updated_20221129/$i.id \
  --name common \
  --hwe_filter 5e-08 \
  --container /mnt/vast/hpc/csg/containers/lmm.sif 
done

In [ ]:
# merge two data-sets
bash: container= '/mnt/mfs/statgen/containers/lmm.sif'
for i in African European Hispanic; do
    plink --bfile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno/merge_adsp.$i.filtered.related.filtered.extracted \
         --bmerge /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno/merge_adsp.$i.filtered.unrelated_common.filtered.bed \
                  /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno/merge_adsp.$i.filtered.unrelated_common.filtered.bim \
                  /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno/merge_adsp.$i.filtered.unrelated_common.filtered.fam \
        --make-bed --out /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno/adsp_geno_qced.$i
done

In [9]:
##in order to run gemma, pheno column in fam file can not be missing, so make modification on fam file.
data=read.table('/mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno/adsp_geno_qced.African.fam',header=F)
data[,6]=5
write.table(data,'/mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno/adsp_geno_qced.African.fam',col.names=F,row.names=F,quote=F,sep = '\t')

data1=read.table('/mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno/adsp_geno_qced.Hispanic.fam',header=F)
data1[,6]=5
write.table(data1,'/mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno/adsp_geno_qced.Hispanic.fam',col.names=F,row.names=F,quote=F,sep = '\t')

data2=read.table('/mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno/adsp_geno_qced.European.fam',header=F)
data2[,6]=5
write.table(data2,'/mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno/adsp_geno_qced.European.fam',col.names=F,row.names=F,quote=F,sep = '\t')

 ## kinship matrix

In [10]:
ml GEMMA
for i in African European Hispanic; do
gemma \
  -bfile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno/adsp_geno_qced.$i \
  -gk 2 \
  -o $i \
  -outdir /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno/kinship
done

GEMMA 0.98.5 (2021-08-25) by Xiang Zhou, Pjotr Prins and team (C) 2012-2021
Reading Files ... 
## number of total individuals = 3830
## number of analyzed individuals = 3830
## number of covariates = 1
## number of phenotypes = 1
## number of total SNPs/var        =    98807
## number of analyzed SNPs         =    98321
Calculating Relatedness Matrix ... 
================================================== 100%
**** INFO: Done.
GEMMA 0.98.5 (2021-08-25) by Xiang Zhou, Pjotr Prins and team (C) 2012-2021
Reading Files ... 
## number of total individuals = 11018
## number of analyzed individuals = 11018
## number of covariates = 1
## number of phenotypes = 1
## number of total SNPs/var        =    46306
## number of analyzed SNPs         =    46201
Calculating Relatedness Matrix ... 
================================================== 100%
**** INFO: Done.
GEMMA 0.98.5 (2021-08-25) by Xiang Zhou, Pjotr Prins and team (C) 2012-2021
Reading Files ... 
## number of total individuals = 2164
## 

## Null models

In [ ]:
#MODEL ONE
for i in African European Hispanic; do
module load Singularity
sos run ~/project2022/bioworkflows/GWAS/LMM.ipynb null_model \
--cwd /mnt/mfs/statgen/alzheimers-family/ADSP_2023/null_model \
--bfile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno/adsp_geno_qced.$i.fam \
--phenoFile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/PCA_QC/$i.pca.projected_outlier_removed.txt \
--grmFile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno/kinship/$i.sXX.txt \
--phenoCol AD \
--container_lmm /mnt/vast/hpc/csg/containers/lmm.sif \
--covarCol Sex \
--qCovarCol Age PC1 PC2 PC3 \
--model_name noAPOE4
done

In [ ]:
#MODEL two
for i in African European Hispanic; do
module load Singularity
sos run ~/project2022/bioworkflows/GWAS/LMM.ipynb null_model \
--cwd /mnt/mfs/statgen/alzheimers-family/ADSP_2023/null_model \
--bfile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno/adsp_geno_qced.$i.fam \
--phenoFile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/PCA_QC/$i.pca.projected_outlier_removed.txt \
--grmFile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno/kinship/$i.sXX.txt \
--phenoCol AD \
--container_lmm /mnt/vast/hpc/csg/containers/lmm.sif \
--covarCol Sex APOE \
--qCovarCol Age PC1 PC2 PC3 \
--model_name APOE4
done

## Analyses

## model 1 : age, sex and 3 PCs as covariate

In [ ]:
#!/bin/sh
#$ -l h_rt=2:00:00
#$ -l h_vmem=40G
#$ -N gmmat_adsp_noAPOE4
#$ -o /mnt/mfs/statgen/alzheimers-family/ADSP_2023/gmmat/noAPOE-$JOB_ID.out
#$ -e /mnt/mfs/statgen/alzheimers-family/ADSP_2023/gmmat/noAPOE-$JOB_ID.err  
#$ -j y
#$ -S /bin/bash

export PATH=$HOME/miniconda3/bin:$PATH

module load PLINK 

module load Singularity/3.5.3

for i in African White Hispanic; do
sos run ~/project2022/bioworkflows/GWAS/LMM.ipynb GMMAT \
--cwd /mnt/mfs/statgen/alzheimers-family/ADSP_2023/gmmat/noapoe/ \
--bfile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno/adsp_geno_qced.$i.bed \
--null_model /mnt/mfs/statgen/alzheimers-family/ADSP_2023/null_model/adsp_geno_qced.$i.ADSP.$i.noAPOE.rds \
--phenoFile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/PCA_QC/$i.pca.projected_outlier_removed.txt \
--formatFile ~/project2022/bioworkflows/GWAS/data/gmmat_template.yml \
--label_annotate SNP \
--phenoCol AD \
--covarMaxLevels 10 \
--numThreads 20 \
--bgenMinMAF 0.01 \
--container_lmm /mnt/vast/hpc/csg/containers/lmm.sif \
--container_marp /mnt/vast/hpc/csg/containers/marp.sif \
--geno_filter 0.01 \
--nperbatch 100
done

## model 2 : age, sex, APOE4 and 3 PCs as covariate

In [ ]:
#!/bin/sh
#$ -l h_rt=2:00:00
#$ -l h_vmem=40G
#$ -N gmmat_adsp_noAPOE4
#$ -o /mnt/mfs/statgen/alzheimers-family/ADSP_2023/gmmat/APOE-$JOB_ID.out
#$ -e /mnt/mfs/statgen/alzheimers-family/ADSP_2023/gmmat/APOE-$JOB_ID.err  
#$ -j y
#$ -S /bin/bash

export PATH=$HOME/miniconda3/bin:$PATH

module load PLINK 

module load Singularity/3.5.3

for i in African White Hispanic; do
sos run ~/project2022/bioworkflows/GWAS/LMM.ipynb GMMAT \
--cwd /mnt/mfs/statgen/alzheimers-family/ADSP_2023/gmmat/apoe/ \
--bfile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno/adsp_geno_qced.$i.bed \
--null_model /mnt/mfs/statgen/alzheimers-family/ADSP_2023/null_model/adsp_geno_qced.$i.ADSP.$i.APOE.rds \
--phenoFile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/PCA_QC/$i.pca.projected_outlier_removed.txt \
--formatFile ~/project2022/bioworkflows/GWAS/data/gmmat_template.yml \
--label_annotate SNP \
--phenoCol AD \
--covarMaxLevels 10 \
--numThreads 20 \
--bgenMinMAF 0.01 \
--container_lmm /mnt/vast/hpc/csg/containers/lmm.sif \
--container_marp /mnt/vast/hpc/csg/containers/marp.sif \
--geno_filter 0.01 \
--nperbatch 100
done

### Meta-analyes

#### model 1 : no APOE4

In [ ]:
SCHEME STDERR
MARKER   SNP
WEIGHT   N
ALLELE   A2 A1
EFFECT   BHAT
STDERR   SBHAT
PVAL     PVAL
FREQLABEL AF
GENOMICCONTROL ON
AVERAGEFREQ ON
MINMAXFREQ ON

PROCESS /mnt/mfs/statgen/alzheimers-family/ADSP_2023/gmmat/noapoe/adsp_geno_qced.European.European.pca.projected_outlier_removed.gmmat.score.txt.gz
PROCESS /mnt/mfs/statgen/alzheimers-family/ADSP_2023/gmmat/noapoe/adsp_geno_qced.Hispanic.Hispanic.pca.projected_outlier_removed.gmmat.score.txt.gz
PROCESS /mnt/mfs/statgen/alzheimers-family/ADSP_2023/gmmat/noapoe/adsp_geno_qced.African.African.pca.projected_outlier_removed.gmmat.score.txt.gz


OUTFILE /mnt/mfs/statgen/alzheimers-family/ADSP_2023/gmmat/noapoe/ADSP.GMMAT_noAPOE4_META .TXT
ANALYZE
ANALYZE HETEROGENEITY

In [1]:
library(stringr) # to replace strings
library(tidyr) # to get  separate function separate the marker name 
library(data.table)
library(dplyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
meta_noAPOE <- read.table('/mnt/mfs/statgen/alzheimers-family/ADSP_2023/gmmat/noapoe/ADSP.GMMAT_noAPOE4_META2.TXT', header = T, sep = '\t')
meta_noAPOE <- meta_noAPOE %>% mutate (SNP = MarkerName)
meta_noAPOE <- meta_noAPOE %>% mutate(across('MarkerName', str_replace_all, '_', ':'))
meta_noAPOE <- meta_noAPOE %>% separate(MarkerName, c('CHR','POS','REF', 'ALT'),  sep = ':')
meta_noAPOE <- meta_noAPOE %>% mutate(across('CHR', str_replace, 'chr', ''))
meta_noAPOE$POS <-  as.numeric(meta_noAPOE$POS)
meta_noAPOE$CHR <-  as.numeric(meta_noAPOE$CHR)
write.table(meta_noAPOE,'/mnt/mfs/statgen/alzheimers-family/ADSP_2023/gmmat/noapoe/meta.gmmat_noAPOE4.txt', sep = '\t', quote = F, col.names = T, row.names = F)
lambda <- median(qchisq(meta_noAPOE$P.value, df=1, lower.tail=FALSE)) / qchisq(0.5, 1)

In [3]:
lambda

[1] 0.9934837

#### model 2 : with APOE4

In [ ]:
SCHEME STDERR
MARKER   SNP
WEIGHT   N
ALLELE   A2 A1
EFFECT   BHAT
STDERR   SBHAT
PVAL     PVAL
FREQLABEL AF
GENOMICCONTROL ON
AVERAGEFREQ ON
MINMAXFREQ ON

PROCESS /mnt/mfs/statgen/alzheimers-family/ADSP_2023/gmmat/apoe/adsp_geno_qced.European.European.pca.projected_outlier_removed.gmmat.score.txt.gz
PROCESS /mnt/mfs/statgen/alzheimers-family/ADSP_2023/gmmat/apoe/adsp_geno_qced.Hispanic.Hispanic.pca.projected_outlier_removed.gmmat.score.txt.gz
PROCESS /mnt/mfs/statgen/alzheimers-family/ADSP_2023/gmmat/apoe/adsp_geno_qced.African.African.pca.projected_outlier_removed.gmmat.score.txt.gz

OUTFILE /mnt/mfs/statgen/alzheimers-family/ADSP_2023/gmmat/apoe/ADSP.GMMAT_APOE4_META .TXT
ANALYZE
ANALYZE HETEROGENEITY

In [8]:
meta_APOE <- read.table('/mnt/mfs/statgen/alzheimers-family/ADSP_2023/gmmat/apoe/ADSP.GMMAT_APOE4_META2.TXT', header = T, sep = '\t')
meta_APOE <- meta_APOE %>% mutate (SNP = MarkerName)
meta_APOE <- meta_APOE %>% mutate(across('MarkerName', str_replace_all, '_', ':'))
meta_APOE <- meta_APOE %>% separate(MarkerName, c('CHR','POS','REF', 'ALT'),  sep = ':')
meta_APOE <- meta_APOE %>% mutate(across('CHR', str_replace, 'chr', ''))
meta_APOE$POS <-  as.numeric(meta_APOE$POS)
meta_APOE$CHR <-  as.numeric(meta_APOE$CHR)
write.table(meta_APOE,'/mnt/mfs/statgen/alzheimers-family/ADSP_2023/gmmat/apoe/meta.gmmat_APOE4.txt', sep = '\t', quote = F, col.names = T, row.names = F)
lambda <- median(qchisq(meta_APOE$P.value, df=1, lower.tail=FALSE)) / qchisq(0.5, 1)

In [9]:
lambda

[1] 0.996738

# Rare Variants

## Annotation

In [ ]:
#Annotation
[submit_chrs]
import glob
bim_files = glob.glob("/mnt/mfs/statgen/alzheimers-family/ADSP/geno/*.filtered.bim")
input: bim_files,group_by = 1
bash:expand = TRUE
sos run ~/bioworkflows/variant-annotation/annovar.ipynb annovar \
    --cwd /mnt/mfs/statgen/alzheimers-family/ADSP/smmat/annotation \
    --bim_name ${_input} \
    --humandb /mnt/mfs/statgen/isabelle/REF/humandb \
    --xref_path /mnt/mfs/statgen/isabelle/REF/humandb \
    --numThreads 1 \
    --name_prefix all \
    --container_annovar /mnt/mfs/statgen/containers/gatk4-annovar.sif \
    -c bioworkflows/admin/csg.yml -q csg -s force &> annovar.log

In [22]:
#Annotation
for i in {1..22};do 
  sos run ~/project2022/bioworkflows/variant-annotation/annovar.ipynb annovar \
    --cwd /mnt/mfs/statgen/alzheimers-family/ADSP_2023/annotation \
    --bim_name /mnt/mfs/statgen/alzheimers-family/ADSP/geno/gcad.qc.r2.wes.chr$i.20504.GATK.2020.06.26.biallelic.genotypes.ALL.leftnorm.filtered.bim \
    --humandb /mnt/mfs/statgen/isabelle/REF/humandb \
    --xref_path /mnt/mfs/statgen/isabelle/REF/humandb \
    --numThreads 1 \
    --name_prefix all \
    --container_annovar /mnt/mfs/statgen/containers/gatk4-annovar.sif \
    -c ~/project2022/bioworkflows/admin/csg.yml -s force &> /mnt/mfs/statgen/alzheimers-family/ADSP_2023/annotation/annovar.log
done

In [66]:
#Generate group files for SMMAT
[group]
import glob
csv_files = glob.glob("/mnt/mfs/statgen/alzheimers-family/ADSP_2023/annotation/*.csv")
input: csv_files,group_by = 1
output: f'{_input:nnn}'
task: trunk_workers = 1, trunk_size = 1, walltime = '10h', mem = '32G', cores = 1, tags = f'{step_name}_{_output:bn}'
R:  container= '/mnt/mfs/statgen/containers/lmm.sif',expand = "${ }"
    library('dplyr')
    data = read.csv('${_input}')
    data$Gene.refGene = sub("\\;.*", "", data$Gene.refGene)
    df = data[, c('Gene.refGene', 'Chr','Start','Alt','Ref','ExonicFunc.refGene','Func.refGene','AF_afr','AF_nfe','AF_amr','AF_afr.1','AF_nfe.1','AF_amr.1','REVEL_score')]
    df[,15]=1
    df$REVEL_score = as.numeric(as.character(df$REVEL_score))
    df$AF_afr = as.numeric(as.character(df$AF_afr))
    df$AF_nfe = as.numeric(as.character(df$AF_nfe))
    df$AF_amr = as.numeric(as.character(df$AF_amr))
    model1=df%>%filter((ExonicFunc.refGene %in% c('startloss','stopgain')) | (Func.refGene %in% c('splicing', 'exonic;splicing', 'ncRNA_exonic;splicing', 'ncRNA_splicing')))
    model2=df%>%filter((ExonicFunc.refGene %in% c('startloss','stopgain')) | (Func.refGene %in% c('splicing', 'exonic;splicing', 'ncRNA_exonic;splicing', 'ncRNA_splicing')) | (ExonicFunc.refGene=='nonsynonymous SNV' & REVEL_score>=0.5))
    model3=df%>%filter((ExonicFunc.refGene %in% c('nonsynonymous SNV','startloss','stopgain')) | (Func.refGene %in% c('splicing', 'exonic;splicing', 'ncRNA_exonic;splicing', 'ncRNA_splicing')))

    #African
    model1_a1=model1[which((model1$AF_afr<=0.01&model1$AF_afr.1<=0.01)|(model1$AF_afr>0.99&model1$AF_afr.1>0.99)|is.na(model1$AF_afr)),c(1, 2, 3, 4, 5, 15)]
    model1_a5=model1[which((model1$AF_afr<=0.05&model1$AF_afr.1<=0.05)|is.na(model1$AF_afr)),c(1, 2, 3, 4, 5, 15)]
    model2_a1=model2[which((model2$AF_afr<=0.01&model2$AF_afr.1<=0.01)|(model2$AF_afr>0.99&model2$AF_afr.1>0.99)|is.na(model2$AF_afr)),c(1, 2, 3, 4, 5, 15)]
    model2_a5=model2[which((model2$AF_afr<=0.05&model2$AF_afr.1<=0.05)|is.na(model2$AF_afr)),c(1, 2, 3, 4, 5, 15)]
    model3_a1=model3[which((model3$AF_afr<=0.01&model3$AF_afr.1<=0.01)|(model3$AF_afr>0.99&model3$AF_afr.1>0.99)|is.na(model3$AF_afr)),c(1, 2, 3, 4, 5, 15)]
    model3_a5=model3[which((model3$AF_afr<=0.05&model3$AF_afr.1<=0.05)|is.na(model3$AF_afr)),c(1, 2, 3, 4, 5, 15)]

    write.table(model1_a1, '${_output}.African.model1.af1.group.file', col.names = F, row.names = F, quote = F, sep = '\t')
    write.table(model1_a5, '${_output}.African.model1.af5.group.file', col.names = F, row.names = F, quote = F, sep = '\t')
    write.table(model2_a1, '${_output}.African.model2.af1.group.file', col.names = F, row.names = F, quote = F, sep = '\t')
    write.table(model2_a5, '${_output}.African.model2.af5.group.file', col.names = F, row.names = F, quote = F, sep = '\t')
    write.table(model3_a1, '${_output}.African.model3.af1.group.file', col.names = F, row.names = F, quote = F, sep = '\t')
    write.table(model3_a5, '${_output}.African.model3.af5.group.file', col.names = F, row.names = F, quote = F, sep = '\t')

    #European
    model1_e1=model1[which((model1$AF_afr<=0.01&model1$AF_nfe.1<=0.01)|(model1$AF_nfe>0.99&model1$AF_nfe.1>0.99)|is.na(model1$AF_nfe)),c(1, 2, 3, 4, 5, 15)]
    model1_e5=model1[which((model1$AF_nfe<=0.05&model1$AF_nfe.1<=0.05)|is.na(model1$AF_nfe)),c(1, 2, 3, 4, 5, 15)]
    model2_e1=model2[which((model2$AF_nfe<=0.01&model2$AF_nfe.1<=0.01)|(model1$AF_nfe>0.99&model2$AF_nfe.2>0.99)|is.na(model2$AF_nfe)),c(1, 2, 3, 4, 5, 15)]
    model2_e5=model2[which((model2$AF_nfe<=0.05&model2$AF_nfe.1<=0.05)|is.na(model2$AF_nfe)),c(1, 2, 3, 4, 5, 15)]
    model3_e1=model3[which((model3$AF_nfe<=0.01&model3$AF_nfe.1<=0.01)|(model3$AF_nfe>0.99&model3$AF_nfe.1>0.99)|is.na(model3$AF_nfe)),c(1, 2, 3, 4, 5, 15)]
    model3_e5=model3[which((model3$AF_nfe<=0.05&model3$AF_nfe.1<=0.05)|is.na(model3$AF_nfe)),c(1, 2, 3, 4, 5, 15)]

    write.table(model1_e1, '${_output}.European.model1.af1.group.file', col.names = F, row.names = F, quote = F, sep = '\t')
    write.table(model1_e5, '${_output}.European.model1.af5.group.file', col.names = F, row.names = F, quote = F, sep = '\t')
    write.table(model2_e1, '${_output}.European.model2.af1.group.file', col.names = F, row.names = F, quote = F, sep = '\t')
    write.table(model2_e5, '${_output}.European.model2.af5.group.file', col.names = F, row.names = F, quote = F, sep = '\t')
    write.table(model3_e1, '${_output}.European.model3.af1.group.file', col.names = F, row.names = F, quote = F, sep = '\t')
    write.table(model3_e5, '${_output}.European.model3.af5.group.file', col.names = F, row.names = F, quote = F, sep = '\t')
  
    #Hispanic
    model1_h1=model1%>%filter((AF_afr<=0.01&AF_afr.1<=0.01)|(AF_afr>0.99&AF_afr.1>0.99)|is.na(AF_afr))%>%filter((AF_nfe<=0.01&AF_nfe.1<=0.01)|(AF_nfe>0.99&AF_nfe.1>0.99)|is.na(AF_nfe))%>%filter((AF_amr<=0.01&AF_amr.1<=0.01)|(AF_amr>0.99&AF_amr.1>0.99)|is.na(AF_amr))%>%select(c(1,2, 3, 4, 5, 15))
    model1_h5=model1%>%filter((AF_afr<=0.05&AF_afr.1<=0.05)|is.na(AF_afr))%>%filter((AF_nfe<=0.05&AF_nfe.1<=0.05)|is.na(AF_nfe))%>%filter((AF_amr<=0.05&AF_amr.1<=0.05)|is.na(AF_amr))%>%select(c(1,2, 3, 4, 5, 15))
    model2_h1=model2%>%filter((AF_afr<=0.01&AF_afr.1<=0.01)|(AF_afr>0.99&AF_afr.1>0.99)|is.na(AF_afr))%>%filter((AF_nfe<=0.01&AF_nfe.1<=0.01)|(AF_nfe>0.99&AF_nfe.1>0.99)|is.na(AF_nfe))%>%filter((AF_amr<=0.01&AF_amr.1<=0.01)|(AF_amr>0.99&AF_amr.1>0.99)|is.na(AF_amr))%>%select(c(1,2, 3, 4, 5, 15))
    model2_h5=model2%>%filter((AF_afr<=0.05&AF_afr.1<=0.05)|is.na(AF_afr))%>%filter((AF_nfe<=0.05&AF_nfe.1<=0.05)|is.na(AF_nfe))%>%filter((AF_amr<=0.05&AF_amr.1<=0.05)|is.na(AF_amr))%>%select(c(1,2, 3, 4, 5, 15))
    model3_h1=model3%>%filter((AF_afr<=0.01&AF_afr.1<=0.01)|(AF_afr>0.99&AF_afr.1>0.99)|is.na(AF_afr))%>%filter((AF_nfe<=0.01&AF_nfe.1<=0.01)|(AF_nfe>0.99&AF_nfe.1>0.99)|is.na(AF_nfe))%>%filter((AF_amr<=0.01&AF_amr.1<=0.01)|(AF_amr>0.99&AF_amr.1>0.99)|is.na(AF_amr))%>%select(c(1,2, 3, 4, 5, 15))
    model3_h5=model3%>%filter((AF_afr<=0.05&AF_afr.1<=0.05)|is.na(AF_afr))%>%filter((AF_nfe<=0.05&AF_nfe.1<=0.05)|is.na(AF_nfe))%>%filter((AF_amr<=0.05&AF_amr.1<=0.05)|is.na(AF_amr))%>%select(c(1,2, 3, 4, 5, 15))
 
    write.table(model1_h1, '${_output}.Hispanic.model1.af1.group.file', col.names = F, row.names = F, quote = F, sep = '\t')
    write.table(model1_h5, '${_output}.Hispanic.model1.af5.group.file', col.names = F, row.names = F, quote = F, sep = '\t')
    write.table(model2_h1, '${_output}.Hispanic.model2.af1.group.file', col.names = F, row.names = F, quote = F, sep = '\t')
    write.table(model2_h5, '${_output}.Hispanic.model2.af5.group.file', col.names = F, row.names = F, quote = F, sep = '\t')
    write.table(model3_h1, '${_output}.Hispanic.model3.af1.group.file', col.names = F, row.names = F, quote = F, sep = '\t')
    write.table(model3_h5, '${_output}.Hispanic.model3.af5.group.file', col.names = F, row.names = F, quote = F, sep = '\t')


In [82]:
ml Singularity
sos run ~/project_bst/notebook_Bale/ADSP/annatation_file.ipynb submit_chrs

INFO: Running submit_chrs: Annotation
INFO: submit_chrs (index=4) is completed.
INFO: submit_chrs (index=5) is completed.
INFO: submit_chrs (index=7) is completed.
INFO: submit_chrs (index=6) is completed.
INFO: submit_chrs (index=1) is completed.
INFO: submit_chrs (index=3) is completed.
INFO: submit_chrs (index=2) is completed.
INFO: submit_chrs (index=9) is completed.
INFO: submit_chrs (index=13) is completed.
INFO: submit_chrs (index=0) is completed.
INFO: submit_chrs (index=12) is completed.
INFO: submit_chrs (index=14) is completed.
INFO: submit_chrs (index=8) is completed.
INFO: submit_chrs (index=10) is completed.
INFO: submit_chrs (index=16) is completed.
INFO: submit_chrs (index=15) is completed.
INFO: submit_chrs (index=17) is completed.
INFO: submit_chrs (index=20) is completed.
INFO: submit_chrs (index=18) is completed.
INFO: submit_chrs (index=19) is completed.
INFO: submit_chrs (index=11) is completed.
INFO: submit_chrs (index=21) is completed.
INFO: Workflow submit_chrs

In [ ]:
#Generate group files for SMMAT

    data = read.csv('/mnt/mfs/statgen/alzheimers-family/ADSP_2023/annotation/gcad.qc.r2.wes.chr11.20504.GATK.2020.06.26.biallelic.genotypes.ALL.leftnorm.filtered.hg38.hg38_multianno.csv',sep = '\t')
    data$Gene.refGene = sub("\\;.*", "", data$Gene.refGene)
    df = data[, c('Gene.refGene', 'Chr','Start','Alt','Ref','ExonicFunc.refGene','Func.refGene','AF_afr','AF_nfe','AF_amr','AF_afr.1','AF_nfe.1','AF_amr.1','REVEL_score')]
    df[,15]=1
    df$REVEL_score = as.numeric(as.character(df$REVEL_score))
    df$AF_afr = as.numeric(as.character(df$AF_afr))
    df$AF_nfe = as.numeric(as.character(df$AF_nfe))
    df$AF_amr = as.numeric(as.character(df$AF_amr))
    df$AF_afr.1 = as.numeric(as.character(df$AF_afr.1))
    df$AF_nfe.1 = as.numeric(as.character(df$AF_nfe.1))
    df$AF_amr.1 = as.numeric(as.character(df$AF_amr.1))
    model1=df%>%filter((ExonicFunc.refGene %in% c('startloss','stopgain')) | (Func.refGene %in% c('splicing', 'exonic;splicing', 'ncRNA_exonic;splicing', 'ncRNA_splicing')))
    model2=df%>%filter((ExonicFunc.refGene %in% c('startloss','stopgain')) | (Func.refGene %in% c('splicing', 'exonic;splicing', 'ncRNA_exonic;splicing', 'ncRNA_splicing')) | (ExonicFunc.refGene=='nonsynonymous SNV' & REVEL_score>=0.5))
    model3=df%>%filter((ExonicFunc.refGene %in% c('nonsynonymous SNV','startloss','stopgain')) | (Func.refGene %in% c('splicing', 'exonic;splicing', 'ncRNA_exonic;splicing', 'ncRNA_splicing')))

    #African
    model1_a1=model1[which((model1$AF_afr<=0.01&model1$AF_afr.1<=0.01)|(model1$AF_afr>0.99&model1$AF_afr.1>0.99)|is.na(model1$AF_afr)),c(1, 2, 3, 4, 5, 15)]
    model1_a5=model1[which((model1$AF_afr<=0.05&model1$AF_afr.1<=0.05)|is.na(model1$AF_afr)),c(1, 2, 3, 4, 5, 15)]
    model2_a1=model2[which((model2$AF_afr<=0.01&model2$AF_afr.1<=0.01)|(model2$AF_afr>0.99&model2$AF_afr.1>0.99)|is.na(model2$AF_afr)),c(1, 2, 3, 4, 5, 15)]
    model2_a5=model2[which((model2$AF_afr<=0.05&model2$AF_afr.1<=0.05)|is.na(model2$AF_afr)),c(1, 2, 3, 4, 5, 15)]
    model3_a1=model3[which((model3$AF_afr<=0.01&model3$AF_afr.1<=0.01)|(model3$AF_afr>0.99&model3$AF_afr.1>0.99)|is.na(model3$AF_afr)),c(1, 2, 3, 4, 5, 15)]
    model3_a5=model3[which((model3$AF_afr<=0.05&model3$AF_afr.1<=0.05)|is.na(model3$AF_afr)),c(1, 2, 3, 4, 5, 15)]

    write.table(model1_a1, paste0('/mnt/mfs/statgen/alzheimers-family/ADSP_2023/annotation/gcad.qc.r2.wes.chr11.20504.GATK.2020.06.26.biallelic.genotypes.ALL.African.model1.af1.group.file'), col.names = F, row.names = F, quote = F, sep = '\t')
    write.table(model1_a5, paste0('/mnt/mfs/statgen/alzheimers-family/ADSP_2023/annotation/gcad.qc.r2.wes.chr11.20504.GATK.2020.06.26.biallelic.genotypes.ALL.African.model1.af5.group.file'), col.names = F, row.names = F, quote = F, sep = '\t')
    write.table(model2_a1, paste0('/mnt/mfs/statgen/alzheimers-family/ADSP_2023/annotation/gcad.qc.r2.wes.chr11.20504.GATK.2020.06.26.biallelic.genotypes.ALL.African.model2.af1.group.file'), col.names = F, row.names = F, quote = F, sep = '\t')
    write.table(model2_a5, paste0('/mnt/mfs/statgen/alzheimers-family/ADSP_2023/annotation/gcad.qc.r2.wes.chr11.20504.GATK.2020.06.26.biallelic.genotypes.ALL.African.model2.af5.group.file'), col.names = F, row.names = F, quote = F, sep = '\t')
    write.table(model3_a1, paste0('/mnt/mfs/statgen/alzheimers-family/ADSP_2023/annotation/gcad.qc.r2.wes.chr11.20504.GATK.2020.06.26.biallelic.genotypes.ALL.African.model3.af1.group.file'), col.names = F, row.names = F, quote = F, sep = '\t')
    write.table(model3_a5, paste0('/mnt/mfs/statgen/alzheimers-family/ADSP_2023/annotation/gcad.qc.r2.wes.chr11.20504.GATK.2020.06.26.biallelic.genotypes.ALL.African.model3.af5.group.file'), col.names = F, row.names = F, quote = F, sep = '\t')

    #European
    model1_e1=model1[which((model1$AF_nfe<=0.01&model1$AF_nfe.1<=0.01)|(model1$AF_nfe>0.99&model1$AF_nfe.1>0.99)|is.na(model1$AF_nfe)),c(1, 2, 3, 4, 5, 15)]
    model1_e5=model1[which((model1$AF_nfe<=0.05&model1$AF_nfe.1<=0.05)|is.na(model1$AF_nfe)),c(1, 2, 3, 4, 5, 15)]
    model2_e1=model2[which((model2$AF_nfe<=0.01&model2$AF_nfe.1<=0.01)|(model1$AF_nfe>0.99&model2$AF_nfe.2>0.99)|is.na(model2$AF_nfe)),c(1, 2, 3, 4, 5, 15)]
    model2_e5=model2[which((model2$AF_nfe<=0.05&model2$AF_nfe.1<=0.05)|is.na(model2$AF_nfe)),c(1, 2, 3, 4, 5, 15)]
    model3_e1=model3[which((model3$AF_nfe<=0.01&model3$AF_nfe.1<=0.01)|(model3$AF_nfe>0.99&model3$AF_nfe.1>0.99)|is.na(model3$AF_nfe)),c(1, 2, 3, 4, 5, 15)]
    model3_e5=model3[which((model3$AF_nfe<=0.05&model3$AF_nfe.1<=0.05)|is.na(model3$AF_nfe)),c(1, 2, 3, 4, 5, 15)]

    write.table(model1_e1, paste0('/mnt/mfs/statgen/alzheimers-family/ADSP_2023/annotation/gcad.qc.r2.wes.chr11.20504.GATK.2020.06.26.biallelic.genotypes.ALL.European.model1.af1.group.file'), col.names = F, row.names = F, quote = F, sep = '\t')
    write.table(model1_e5, paste0('/mnt/mfs/statgen/alzheimers-family/ADSP_2023/annotation/gcad.qc.r2.wes.chr11.20504.GATK.2020.06.26.biallelic.genotypes.ALL.European.model1.af5.group.file'), col.names = F, row.names = F, quote = F, sep = '\t')
    write.table(model2_e1, paste0('/mnt/mfs/statgen/alzheimers-family/ADSP_2023/annotation/gcad.qc.r2.wes.chr11.20504.GATK.2020.06.26.biallelic.genotypes.ALL.European.model2.af1.group.file'), col.names = F, row.names = F, quote = F, sep = '\t')
    write.table(model2_e5, paste0('/mnt/mfs/statgen/alzheimers-family/ADSP_2023/annotation/gcad.qc.r2.wes.chr11.20504.GATK.2020.06.26.biallelic.genotypes.ALL.European.model2.af5.group.file'), col.names = F, row.names = F, quote = F, sep = '\t')
    write.table(model3_e1, paste0('/mnt/mfs/statgen/alzheimers-family/ADSP_2023/annotation/gcad.qc.r2.wes.chr11.20504.GATK.2020.06.26.biallelic.genotypes.ALL.European.model3.af1.group.file'), col.names = F, row.names = F, quote = F, sep = '\t')
    write.table(model3_e5, paste0('/mnt/mfs/statgen/alzheimers-family/ADSP_2023/annotation/gcad.qc.r2.wes.chr11.20504.GATK.2020.06.26.biallelic.genotypes.ALL.European.model3.af5.group.file'), col.names = F, row.names = F, quote = F, sep = '\t')

    #Hispanic
    model1_h1=model1%>%filter((AF_afr<=0.01&AF_afr.1<=0.01)|(AF_afr>0.99&AF_afr.1>0.99)|is.na(AF_afr))%>%filter((AF_nfe<=0.01&AF_nfe.1<=0.01)|(AF_nfe>0.99&AF_nfe.1>0.99)|is.na(AF_nfe))%>%filter((AF_amr<=0.01&AF_amr.1<=0.01)|(AF_amr>0.99&AF_amr.1>0.99)|is.na(AF_amr))%>%select(c(1,2, 3, 4, 5, 15))
    model1_h5=model1%>%filter((AF_afr<=0.05&AF_afr.1<=0.05)|is.na(AF_afr))%>%filter((AF_nfe<=0.05&AF_nfe.1<=0.05)|is.na(AF_nfe))%>%filter((AF_amr<=0.05&AF_amr.1<=0.05)|is.na(AF_amr))%>%select(c(1,2, 3, 4, 5, 15))
    model2_h1=model2%>%filter((AF_afr<=0.01&AF_afr.1<=0.01)|(AF_afr>0.99&AF_afr.1>0.99)|is.na(AF_afr))%>%filter((AF_nfe<=0.01&AF_nfe.1<=0.01)|(AF_nfe>0.99&AF_nfe.1>0.99)|is.na(AF_nfe))%>%filter((AF_amr<=0.01&AF_amr.1<=0.01)|(AF_amr>0.99&AF_amr.1>0.99)|is.na(AF_amr))%>%select(c(1,2, 3, 4, 5, 15))
    model2_h5=model2%>%filter((AF_afr<=0.05&AF_afr.1<=0.05)|is.na(AF_afr))%>%filter((AF_nfe<=0.05&AF_nfe.1<=0.05)|is.na(AF_nfe))%>%filter((AF_amr<=0.05&AF_amr.1<=0.05)|is.na(AF_amr))%>%select(c(1,2, 3, 4, 5, 15))
    model3_h1=model3%>%filter((AF_afr<=0.01&AF_afr.1<=0.01)|(AF_afr>0.99&AF_afr.1>0.99)|is.na(AF_afr))%>%filter((AF_nfe<=0.01&AF_nfe.1<=0.01)|(AF_nfe>0.99&AF_nfe.1>0.99)|is.na(AF_nfe))%>%filter((AF_amr<=0.01&AF_amr.1<=0.01)|(AF_amr>0.99&AF_amr.1>0.99)|is.na(AF_amr))%>%select(c(1,2, 3, 4, 5, 15))
    model3_h5=model3%>%filter((AF_afr<=0.05&AF_afr.1<=0.05)|is.na(AF_afr))%>%filter((AF_nfe<=0.05&AF_nfe.1<=0.05)|is.na(AF_nfe))%>%filter((AF_amr<=0.05&AF_amr.1<=0.05)|is.na(AF_amr))%>%select(c(1,2, 3, 4, 5, 15))
 
    write.table(model1_h1, paste0('/mnt/mfs/statgen/alzheimers-family/ADSP_2023/annotation/gcad.qc.r2.wes.chr11.20504.GATK.2020.06.26.biallelic.genotypes.ALL.Hispanic.model1.af1.group.file'), col.names = F, row.names = F, quote = F, sep = '\t')
    write.table(model1_h5, paste0('/mnt/mfs/statgen/alzheimers-family/ADSP_2023/annotation/gcad.qc.r2.wes.chr11.20504.GATK.2020.06.26.biallelic.genotypes.ALL.Hispanic.model1.af5.group.file'), col.names = F, row.names = F, quote = F, sep = '\t')
    write.table(model2_h1, paste0('/mnt/mfs/statgen/alzheimers-family/ADSP_2023/annotation/gcad.qc.r2.wes.chr11.20504.GATK.2020.06.26.biallelic.genotypes.ALL.Hispanic.model2.af1.group.file'), col.names = F, row.names = F, quote = F, sep = '\t')
    write.table(model2_h5, paste0('/mnt/mfs/statgen/alzheimers-family/ADSP_2023/annotation/gcad.qc.r2.wes.chr11.20504.GATK.2020.06.26.biallelic.genotypes.ALL.Hispanic.model2.af5.group.file'), col.names = F, row.names = F, quote = F, sep = '\t')
    write.table(model3_h1, paste0('/mnt/mfs/statgen/alzheimers-family/ADSP_2023/annotation/gcad.qc.r2.wes.chr11.20504.GATK.2020.06.26.biallelic.genotypes.ALL.Hispanic.model3.af1.group.file'), col.names = F, row.names = F, quote = F, sep = '\t')
    write.table(model3_h5, paste0('/mnt/mfs/statgen/alzheimers-family/ADSP_2023/annotation/gcad.qc.r2.wes.chr11.20504.GATK.2020.06.26.biallelic.genotypes.ALL.Hispanic.model3.af5.group.file'), col.names = F, row.names = F, quote = F, sep = '\t')

In [ ]:
data = read.csv('/mnt/mfs/statgen/alzheimers-family/ADSP_2023/annotation/gcad.qc.r2.wes.chr11.20504.GATK.2020.06.26.biallelic.genotypes.ALL.leftnorm.filtered.hg38.hg38_multianno.csv')
    data$Gene.refGene = sub("\\;.*", "", data$Gene.refGene)
    df = data[, c('Gene.refGene', 'Chr','Start','Alt','Ref','ExonicFunc.refGene','Func.refGene','AF_afr','AF_nfe','AF_amr','AF_afr.1','AF_nfe.1','AF_amr.1','REVEL_score')]
    df[,15]=1

## Generate QCed geno data

In [ ]:
# modify outlier.file to make it have enough tokens
for i in African European Hispanic; do 
awk -F' ' '{print $7" "$7}' /mnt/mfs/statgen/alzheimers-family/ADSP_2023/PCA_QC/$i.pca.projected_outlier_removed.txt > /mnt/mfs/statgen/alzheimers-family/ADSP_2023/PCA_QC/$i.pca.projected_outlier_removed.id
done

In [ ]:
# split the geno file per pop and qc
ml PLINK/2.0
for i in African European Hispanic; do   
 plink2 --bfile /mnt/mfs/statgen/alzheimers-family/ADSP/merge_adsp \
        --geno 0.1 \
        --hwe 5e-08 \
        --keep /mnt/mfs/statgen/alzheimers-family/ADSP_pheno_updated_20221129/$i.id \
        --maf 0.0 \
        --make-bed \ 
        --memory 14400.0 \
        --mind 0.1 \
        --out /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/geno/$i_rare --threads 20
done

In [ ]:
for i in African European Hispanic; do 
plink2 --bfile /mnt/mfs/statgen/alzheimers-family/ADSP/merge_adsp --geno 0.1 --hwe 5e-08  --keep /mnt/mfs/statgen/alzheimers-family/ADSP_2023/PCA_QC/$i.pca.projected_outlier_removed.id --maf 0.0 --make-bed --memory 14400.0 --mind 0.1 --out /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/geno/$i --threads 20
done

In [ ]:
plink --bfile /mnt/mfs/statgen/alzheimers-family/ADSP/merge_adsp --geno 0.1 --hwe 5e-08  --keep /mnt/mfs/statgen/alzheimers-family/ADSP_2023/PCA_QC/African.pca.projected_outlier_removed.id --maf 0.000001 --make-bed --memory 14400.0 --mind 0.1 --out /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/geno/African --threads 20

In [ ]:
# generate chromosome specific genoFile

for chr in {1..22}; do
    plink2 --bfile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/geno/African_rare \
          --chr $chr --make-bed \
          --out /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/geno/African_rare_$chr
done

In [ ]:
for chr in {1..22}; do
    plink2 --bfile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno_file/European \
          --chr $chr --make-bed \
          --out /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno_file/European_$chr
    done

In [ ]:
for chr in {1..22}; do
    plink2 --bfile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/geno/Hispanic_rare \
          --chr $chr --make-bed \
          --out /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/geno/Hispanic_rare_$chr
done

# Association tests

## Non-Hispanic Whites

### Model1

In [ ]:
#!/bin/sh
#$ -l h_rt=6:00:00
#$ -l h_vmem=30G
#$ -N smmat_ADSP
#$ -o /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/noapoe/af0.01/model1_skat/af0.01_model1_European-$JOB_ID.out
#$ -e /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/noapoe/af0.01/model1_skat/af0.01_model1_European-$JOB_ID.err  
#$ -j y
#$ -S /bin/bash

export PATH=$HOME/miniconda3/bin:$PATH

module load Singularity/3.5.3

## Run the SMMAT
# with no APOE4 adjustment

sos run ~/project2022/bioworkflows/GWAS/LMM2.ipynb SMMAT \
--cwd  /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/noapoe/af0.01/model1_skat/ \
--groupFile /mnt/mfs/statgen/alzheimers-family/ADSP/smmat/annotation/gcad.qc.r2.wes.chr{1..22}.20504.GATK.2020.06.26.biallelic.genotypes.ALL.leftnorm.filtered.European.model1.af1.group.file \
--posFile /mnt/mfs/statgen/alzheimers-family/ADSP/smmat/annotation/gcad.qc.r2.wes.chr{1..22}.20504.GATK.2020.06.26.biallelic.genotypes.ALL.leftnorm.filtered.hg38.hg38_multianno.csv  \
--bfile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno_file/European_{1..22}.bed \
--null_model /mnt/mfs/statgen/alzheimers-family/ADSP_2023/null_model/adsp_geno_qced.European.European.pca.projected_outlier_removed.noAPOE4.rds \
--grmFile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno/kinship/European.sXX.txt \
--phenoFile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/PCA_QC/European.pca.projected_outlier_removed.txt \
--formatFile   ~/project2022/bioworkflows/GWAS/data/smmat_template_2.yml \
--phenoCol AD \
--label_annotate SNP \
--maf_max_filter 1 \
--covarMaxLevels 10 \
--numThreads 1 \
--bgenMinMAF 0.0 \
--container_lmm /mnt/vast/hpc/csg/containers/lmm.sif \
--container_marp /mnt/vast/hpc/csg/containers/marp.sif \
--geno_filter 0.01 \
--nperbatch 100 \
-c ~/project2022/bioworkflows/admin/csg.yml -s force &> /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/noapoe/af0.01/model1_skat/af0.01_model1_European.log


# with APOE4 adjustment


sos run ~/project2022/bioworkflows/GWAS/LMM2.ipynb SMMAT \
--cwd  /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/apoe/af0.01/model1_skat/ \
--groupFile /mnt/mfs/statgen/alzheimers-family/ADSP/smmat/annotation/gcad.qc.r2.wes.chr{1..22}.20504.GATK.2020.06.26.biallelic.genotypes.ALL.leftnorm.filtered.European.model1.af1.group.file \
--posFile /mnt/mfs/statgen/alzheimers-family/ADSP/smmat/annotation/gcad.qc.r2.wes.chr{1..22}.20504.GATK.2020.06.26.biallelic.genotypes.ALL.leftnorm.filtered.hg38.hg38_multianno.csv  \
--bfile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno_file/European_{1..22}.bed \
--null_model /mnt/mfs/statgen/alzheimers-family/ADSP_2023/null_model/adsp_geno_qced.European.European.pca.projected_outlier_removed.APOE4.rds \
--grmFile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno/kinship/European.sXX.txt \
--phenoFile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/PCA_QC/European.pca.projected_outlier_removed.txt \
--formatFile   ~/project2022/bioworkflows/GWAS/data/smmat_template_2.yml \
--phenoCol AD \
--label_annotate SNP \
--maf_max_filter 1 \
--covarMaxLevels 10 \
--numThreads 1 \
--bgenMinMAF 0.0 \
--container_lmm /mnt/vast/hpc/csg/containers/lmm.sif \
--container_marp /mnt/vast/hpc/csg/containers/marp.sif \
--geno_filter 0.01 \
--nperbatch 100 \
-c ~/project2022/bioworkflows/admin/csg.yml -s force &> /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/apoe/af0.01/model1_skat/af0.01_model1_European.log


### Model2

In [ ]:
#!/bin/sh
#$ -l h_rt=8:00:00
#$ -l h_vmem=40G
#$ -N smmat_ADSP
#$ -o /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/noapoe/af0.01/model2_skat/af0.01_model2_European-$JOB_ID.out
#$ -e /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/noapoe/af0.01/model2_skat/af0.01_model2_European-$JOB_ID.err  
#$ -j y
#$ -S /bin/bash

export PATH=$HOME/miniconda3/bin:$PATH

module load Singularity/3.5.3

## Run the SMMAT
# with no APOE4 adjustment

sos run ~/project2022/bioworkflows/GWAS/LMM2.ipynb SMMAT \
--cwd  /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/noapoe/af0.01/model2_skat/ \
--groupFile /mnt/mfs/statgen/alzheimers-family/ADSP/smmat/annotation/gcad.qc.r2.wes.chr{1..22}.20504.GATK.2020.06.26.biallelic.genotypes.ALL.leftnorm.filtered.European.model2.af1.group.file \
--posFile /mnt/mfs/statgen/alzheimers-family/ADSP/smmat/annotation/gcad.qc.r2.wes.chr{1..22}.20504.GATK.2020.06.26.biallelic.genotypes.ALL.leftnorm.filtered.hg38.hg38_multianno.csv  \
--bfile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno_file/European_{1..22}.bed \
--null_model /mnt/mfs/statgen/alzheimers-family/ADSP_2023/null_model/adsp_geno_qced.European.European.pca.projected_outlier_removed.noAPOE4.rds \
--grmFile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno/kinship/European.sXX.txt \
--phenoFile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/PCA_QC/European.pca.projected_outlier_removed.txt \
--formatFile   ~/project2022/bioworkflows/GWAS/data/smmat_template_2.yml \
--phenoCol AD \
--label_annotate SNP \
--maf_max_filter 1 \
--covarMaxLevels 10 \
--numThreads 1 \
--bgenMinMAF 0.0 \
--container_lmm /mnt/vast/hpc/csg/containers/lmm.sif \
--container_marp /mnt/vast/hpc/csg/containers/marp.sif \
--geno_filter 0.01 \
--nperbatch 100 \
-c ~/project2022/bioworkflows/admin/csg.yml -s force &> /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/noapoe/af0.01/model2_skat/smmat_European_noapoe.log


# with APOE4 adjustment


sos run ~/project2022/bioworkflows/GWAS/LMM2.ipynb SMMAT \
--cwd  /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/apoe/af0.01/model2_skat/ \
--groupFile /mnt/mfs/statgen/alzheimers-family/ADSP/smmat/annotation/gcad.qc.r2.wes.chr{1..22}.20504.GATK.2020.06.26.biallelic.genotypes.ALL.leftnorm.filtered.European.model2.af1.group.file \
--posFile /mnt/mfs/statgen/alzheimers-family/ADSP/smmat/annotation/gcad.qc.r2.wes.chr{1..22}.20504.GATK.2020.06.26.biallelic.genotypes.ALL.leftnorm.filtered.hg38.hg38_multianno.csv  \
--bfile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno_file/European_{1..22}.bed \
--null_model /mnt/mfs/statgen/alzheimers-family/ADSP_2023/null_model/adsp_geno_qced.European.European.pca.projected_outlier_removed.APOE4.rds \
--grmFile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno/kinship/European.sXX.txt \
--phenoFile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/PCA_QC/European.pca.projected_outlier_removed.txt \
--formatFile   ~/project2022/bioworkflows/GWAS/data/smmat_template_2.yml \
--phenoCol AD \
--label_annotate SNP \
--maf_max_filter 1 \
--covarMaxLevels 10 \
--numThreads 1 \
--bgenMinMAF 0.0 \
--container_lmm /mnt/vast/hpc/csg/containers/lmm.sif \
--container_marp /mnt/vast/hpc/csg/containers/marp.sif \
--geno_filter 0.01 \
--nperbatch 100 \
-c ~/project2022/bioworkflows/admin/csg.yml -s force &> /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/apoe/af0.01/model2_skat/smmat_European_apoe.log

## Carribean Hispanics

### Model1

In [ ]:
#!/bin/sh
#$ -l h_rt=2:00:00
#$ -l h_vmem=20G
#$ -N smmat_ADSP
#$ -o /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/noapoe/af0.01/model1_skat/af0.01_model1_Hispanic-$JOB_ID.out
#$ -e /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/noapoe/af0.01/model1_skat/af0.01_model1_Hispanic-$JOB_ID.err  
#$ -j y
#$ -S /bin/bash

export PATH=$HOME/miniconda3/bin:$PATH

module load Singularity/3.5.3

## Run the SMMAT
# with no APOE4 adjustment

sos run ~/project2022/bioworkflows/GWAS/LMM2.ipynb SMMAT \
--cwd  /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/noapoe/af0.01/model1_skat/ \
--groupFile /mnt/mfs/statgen/alzheimers-family/ADSP/smmat/annotation/gcad.qc.r2.wes.chr{1..22}.20504.GATK.2020.06.26.biallelic.genotypes.ALL.leftnorm.filtered.Hispanic.model1.af1.group.file \
--posFile /mnt/mfs/statgen/alzheimers-family/ADSP/smmat/annotation/gcad.qc.r2.wes.chr{1..22}.20504.GATK.2020.06.26.biallelic.genotypes.ALL.leftnorm.filtered.hg38.hg38_multianno.csv  \
--bfile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno_file/Hispanic_{1..22}.bed \
--null_model /mnt/mfs/statgen/alzheimers-family/ADSP_2023/null_model/adsp_geno_qced.Hispanic.Hispanic.pca.projected_outlier_removed.noAPOE4.rds \
--grmFile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno/kinship/Hispanic.sXX.txt \
--phenoFile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/PCA_QC/Hispanic.pca.projected_outlier_removed.txt \
--formatFile   ~/project2022/bioworkflows/GWAS/data/smmat_template_2.yml \
--phenoCol AD \
--label_annotate SNP \
--maf_max_filter 1 \
--covarMaxLevels 10 \
--numThreads 1 \
--bgenMinMAF 0.0 \
--container_lmm /mnt/vast/hpc/csg/containers/lmm.sif \
--container_marp /mnt/vast/hpc/csg/containers/marp.sif \
--geno_filter 0.01 \
--nperbatch 100 \
-c ~/project2022/bioworkflows/admin/csg.yml -s force &> /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/noapoe/af0.01/model1_skat/af0.01_model1_Hispanic.log


# with APOE4 adjustment


sos run ~/project2022/bioworkflows/GWAS/LMM2.ipynb SMMAT \
--cwd  /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/apoe/af0.01/model1_skat/ \
--groupFile /mnt/mfs/statgen/alzheimers-family/ADSP/smmat/annotation/gcad.qc.r2.wes.chr{1..22}.20504.GATK.2020.06.26.biallelic.genotypes.ALL.leftnorm.filtered.Hispanic.model1.af1.group.file \
--posFile /mnt/mfs/statgen/alzheimers-family/ADSP/smmat/annotation/gcad.qc.r2.wes.chr{1..22}.20504.GATK.2020.06.26.biallelic.genotypes.ALL.leftnorm.filtered.hg38.hg38_multianno.csv  \
--bfile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno_file/Hispanic_{1..22}.bed \
--null_model /mnt/mfs/statgen/alzheimers-family/ADSP_2023/null_model/adsp_geno_qced.Hispanic.Hispanic.pca.projected_outlier_removed.APOE4.rds \
--grmFile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno/kinship/Hispanic.sXX.txt \
--phenoFile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/PCA_QC/Hispanic.pca.projected_outlier_removed.txt \
--formatFile   ~/project2022/bioworkflows/GWAS/data/smmat_template_2.yml \
--phenoCol AD \
--label_annotate SNP \
--maf_max_filter 1 \
--covarMaxLevels 10 \
--numThreads 1 \
--bgenMinMAF 0.0 \
--container_lmm /mnt/vast/hpc/csg/containers/lmm.sif \
--container_marp /mnt/vast/hpc/csg/containers/marp.sif \
--geno_filter 0.01 \
--nperbatch 100 \
-c ~/project2022/bioworkflows/admin/csg.yml -s force &> /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/apoe/af0.01/model1_skat/af0.01_model1_Hispanic.log

### Model2

In [ ]:
#!/bin/sh
#$ -l h_rt=6:00:00
#$ -l h_vmem=20G
#$ -N smmat_ADSP
#$ -o /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/noapoe/af0.01/model2_skat/af0.01_model2_Hispanic-$JOB_ID.out
#$ -e /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/noapoe/af0.01/model2_skat/af0.01_model2_Hispanic-$JOB_ID.err  
#$ -j y
#$ -S /bin/bash

export PATH=$HOME/miniconda3/bin:$PATH

module load Singularity/3.5.3

## Run the SMMAT
# with no APOE4 adjustment

sos run ~/project2022/bioworkflows/GWAS/LMM2.ipynb SMMAT \
--cwd  /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/noapoe/af0.01/model2_skat/ \
--groupFile /mnt/mfs/statgen/alzheimers-family/ADSP/smmat/annotation/gcad.qc.r2.wes.chr{1..22}.20504.GATK.2020.06.26.biallelic.genotypes.ALL.leftnorm.filtered.Hispanic.model2.af1.group.file \
--posFile /mnt/mfs/statgen/alzheimers-family/ADSP/smmat/annotation/gcad.qc.r2.wes.chr{1..22}.20504.GATK.2020.06.26.biallelic.genotypes.ALL.leftnorm.filtered.hg38.hg38_multianno.csv  \
--bfile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno_file/Hispanic_{1..22}.bed \
--null_model /mnt/mfs/statgen/alzheimers-family/ADSP_2023/null_model/adsp_geno_qced.Hispanic.Hispanic.pca.projected_outlier_removed.noAPOE4.rds \
--grmFile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno/kinship/Hispanic.sXX.txt \
--phenoFile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/PCA_QC/Hispanic.pca.projected_outlier_removed.txt \
--formatFile   ~/project2022/bioworkflows/GWAS/data/smmat_template_2.yml \
--phenoCol AD \
--label_annotate SNP \
--maf_max_filter 1 \
--covarMaxLevels 10 \
--numThreads 1 \
--bgenMinMAF 0.0 \
--container_lmm /mnt/vast/hpc/csg/containers/lmm.sif \
--container_marp /mnt/vast/hpc/csg/containers/marp.sif \
--geno_filter 0.01 \
--nperbatch 100 \
-c ~/project2022/bioworkflows/admin/csg.yml -s force &> /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/noapoe/af0.01/model2_skat/smmat_Hispanic_noapoe.log


# with APOE4 adjustment


sos run ~/project2022/bioworkflows/GWAS/LMM2.ipynb SMMAT \
--cwd  /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/apoe/af0.01/model2_skat/ \
--groupFile /mnt/mfs/statgen/alzheimers-family/ADSP/smmat/annotation/gcad.qc.r2.wes.chr{1..22}.20504.GATK.2020.06.26.biallelic.genotypes.ALL.leftnorm.filtered.Hispanic.model2.af1.group.file \
--posFile /mnt/mfs/statgen/alzheimers-family/ADSP/smmat/annotation/gcad.qc.r2.wes.chr{1..22}.20504.GATK.2020.06.26.biallelic.genotypes.ALL.leftnorm.filtered.hg38.hg38_multianno.csv  \
--bfile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno_file/Hispanic_{1..22}.bed \
--null_model /mnt/mfs/statgen/alzheimers-family/ADSP_2023/null_model/adsp_geno_qced.Hispanic.Hispanic.pca.projected_outlier_removed.APOE4.rds \
--grmFile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno/kinship/Hispanic.sXX.txt \
--phenoFile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/PCA_QC/Hispanic.pca.projected_outlier_removed.txt \
--formatFile   ~/project2022/bioworkflows/GWAS/data/smmat_template_2.yml \
--phenoCol AD \
--label_annotate SNP \
--maf_max_filter 1 \
--covarMaxLevels 10 \
--numThreads 1 \
--bgenMinMAF 0.0 \
--container_lmm /mnt/vast/hpc/csg/containers/lmm.sif \
--container_marp /mnt/vast/hpc/csg/containers/marp.sif \
--geno_filter 0.01 \
--nperbatch 100 \
-c ~/project2022/bioworkflows/admin/csg.yml -s force &> /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/apoe/af0.01/model2_skat/smmat_Hispanic_apoe.log

## African-Americans

### Model1

In [ ]:
#!/bin/sh
#$ -l h_rt=3:00:00
#$ -l h_vmem=20G
#$ -N smmat_ADSP
#$ -o /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/noapoe/af0.01/model1_skat/af0.01_model1_African-$JOB_ID.out
#$ -e /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/noapoe/af0.01/model1_skat/af0.01_model1_African-$JOB_ID.err  
#$ -j y
#$ -S /bin/bash

export PATH=$HOME/miniconda3/bin:$PATH

module load Singularity/3.5.3

## Run the SMMAT
# with no APOE4 adjustment
sos run ~/project2022/bioworkflows/GWAS/LMM2.ipynb SMMAT \
--cwd  /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/noapoe/af0.01/model1_skat/ \
--groupFile /mnt/mfs/statgen/alzheimers-family/ADSP/smmat/annotation/gcad.qc.r2.wes.chr{1..22}.20504.GATK.2020.06.26.biallelic.genotypes.ALL.leftnorm.filtered.African.model1.af1.group.file \
--posFile /mnt/mfs/statgen/alzheimers-family/ADSP/smmat/annotation/gcad.qc.r2.wes.chr{1..22}.20504.GATK.2020.06.26.biallelic.genotypes.ALL.leftnorm.filtered.hg38.hg38_multianno.csv  \
--bfile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno_file/African_{1..22}.bed \
--null_model /mnt/mfs/statgen/alzheimers-family/ADSP_2023/null_model/adsp_geno_qced.African.African.pca.projected_outlier_removed.noAPOE4.rds \
--grmFile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno/kinship/African.sXX.txt \
--phenoFile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/PCA_QC/African.pca.projected_outlier_removed.txt \
--formatFile   ~/project2022/bioworkflows/GWAS/data/smmat_template_2.yml \
--phenoCol AD \
--label_annotate SNP \
--maf_max_filter 1 \
--covarMaxLevels 10 \
--numThreads 1 \
--bgenMinMAF 0.0 \
--container_lmm /mnt/vast/hpc/csg/containers/lmm.sif \
--container_marp /mnt/vast/hpc/csg/containers/marp.sif \
--geno_filter 0.01 \
--nperbatch 100 \
-c ~/project2022/bioworkflows/admin/csg.yml -s force &> /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/noapoe/af0.01/model1_skat/af0.01_model1_African.log


# with APOE4 adjustment

sos run ~/project2022/bioworkflows/GWAS/LMM2.ipynb SMMAT \
--cwd  /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/apoe/af0.01/model1_skat/ \
--groupFile /mnt/mfs/statgen/alzheimers-family/ADSP/smmat/annotation/gcad.qc.r2.wes.chr{1..22}.20504.GATK.2020.06.26.biallelic.genotypes.ALL.leftnorm.filtered.African.model1.af1.group.file \
--posFile /mnt/mfs/statgen/alzheimers-family/ADSP/smmat/annotation/gcad.qc.r2.wes.chr{1..22}.20504.GATK.2020.06.26.biallelic.genotypes.ALL.leftnorm.filtered.hg38.hg38_multianno.csv  \
--bfile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno_file/African_{1..22}.bed \
--null_model /mnt/mfs/statgen/alzheimers-family/ADSP_2023/null_model/adsp_geno_qced.African.African.pca.projected_outlier_removed.APOE4.rds \
--grmFile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno/kinship/African.sXX.txt \
--phenoFile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/PCA_QC/African.pca.projected_outlier_removed.txt \
--formatFile   ~/project2022/bioworkflows/GWAS/data/smmat_template_2.yml \
--phenoCol AD \
--label_annotate SNP \
--maf_max_filter 1 \
--covarMaxLevels 10 \
--numThreads 1 \
--bgenMinMAF 0.0 \
--container_lmm /mnt/vast/hpc/csg/containers/lmm.sif \
--container_marp /mnt/vast/hpc/csg/containers/marp.sif \
--geno_filter 0.01 \
--nperbatch 100 \
-c ~/project2022/bioworkflows/admin/csg.yml -s force &> /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/apoe/af0.01/model1_skat/af0.01_model1_African.log

### Model2

In [ ]:
#!/bin/sh
#$ -l h_rt=6:00:00
#$ -l h_vmem=20G
#$ -N smmat_ADSP
#$ -o /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/apoe/af0.01/model2_skat/af0.01_model2_African-$JOB_ID.out
#$ -e /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/apoe/af0.01/model2_skat/af0.01_model2_African-$JOB_ID.err  
#$ -j y
#$ -S /bin/bash

export PATH=$HOME/miniconda3/bin:$PATH

module load Singularity/3.5.3

## Run the SMMAT
# with no APOE4 adjustment
  
sos run ~/project2022/bioworkflows/GWAS/LMM2.ipynb SMMAT \
--cwd  /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/noapoe/af0.01/model2_skat/ \
--groupFile /mnt/mfs/statgen/alzheimers-family/ADSP/smmat/annotation/gcad.qc.r2.wes.chr{1..22}.20504.GATK.2020.06.26.biallelic.genotypes.ALL.leftnorm.filtered.African.model2.af1.group.file \
--posFile /mnt/mfs/statgen/alzheimers-family/ADSP/smmat/annotation/gcad.qc.r2.wes.chr{1..22}.20504.GATK.2020.06.26.biallelic.genotypes.ALL.leftnorm.filtered.hg38.hg38_multianno.csv  \
--bfile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno_file/African_{1..22}.bed \
--null_model /mnt/mfs/statgen/alzheimers-family/ADSP_2023/null_model/adsp_geno_qced.African.African.pca.projected_outlier_removed.noAPOE4.rds \
--grmFile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno/kinship/African.sXX.txt \
--phenoFile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/PCA_QC/African.pca.projected_outlier_removed.txt \
--formatFile   ~/project2022/bioworkflows/GWAS/data/smmat_template_2.yml \
--phenoCol AD \
--label_annotate SNP \
--maf_max_filter 1 \
--covarMaxLevels 10 \
--numThreads 1 \
--bgenMinMAF 0.0 \
--container_lmm /mnt/vast/hpc/csg/containers/lmm.sif \
--container_marp /mnt/vast/hpc/csg/containers/marp.sif \
--geno_filter 0.01 \
--nperbatch 100 \
-c ~/project2022/bioworkflows/admin/csg.yml -s force &> /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/noapoe/af0.01/model2_skat/smmat_African_noapoe.log


# with APOE4 adjustment

  
sos run ~/project2022/bioworkflows/GWAS/LMM2.ipynb SMMAT \
--cwd  /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/apoe/af0.01/model2_skat \
--groupFile /mnt/mfs/statgen/alzheimers-family/ADSP/smmat/annotation/gcad.qc.r2.wes.chr{1..22}.20504.GATK.2020.06.26.biallelic.genotypes.ALL.leftnorm.filtered.African.model2.af1.group.file \
--posFile /mnt/mfs/statgen/alzheimers-family/ADSP/smmat/annotation/gcad.qc.r2.wes.chr{1..22}.20504.GATK.2020.06.26.biallelic.genotypes.ALL.leftnorm.filtered.hg38.hg38_multianno.csv  \
--bfile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno_file/African_{1..22}.bed \
--null_model /mnt/mfs/statgen/alzheimers-family/ADSP_2023/null_model/adsp_geno_qced.African.African.pca.projected_outlier_removed.APOE4.rds \
--grmFile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/geno/kinship/African.sXX.txt \
--phenoFile /mnt/mfs/statgen/alzheimers-family/ADSP_2023/PCA_QC/African.pca.projected_outlier_removed.txt \
--formatFile   ~/project2022/bioworkflows/GWAS/data/smmat_template_2.yml \
--phenoCol AD \
--label_annotate SNP \
--maf_max_filter 1 \
--covarMaxLevels 10 \
--numThreads 1 \
--bgenMinMAF 0.0 \
--container_lmm /mnt/vast/hpc/csg/containers/lmm.sif \
--container_marp /mnt/vast/hpc/csg/containers/marp.sif \
--geno_filter 0.01 \
--nperbatch 100 \
-c ~/project2022/bioworkflows/admin/csg.yml -s force &> /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/apoe/af0.01/model2_skat/smmat_African_apoe.log

## Meta-analyses

### Model 1 without adjusting for APOE4

In [ ]:
SCHEME SAMPLESIZE
MARKER  SNP
WEIGHT  N
EFFECT  BETA
PVAL    P


PROCESS /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/noapoe/af0.01/model1_skat/African.pca.projected_outlier_removed_AD.SMMAT.snp_stats.gz
PROCESS /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/noapoe/af0.01/model1_skat/European.pca.projected_outlier_removed_AD.SMMAT.snp_stats.gz
PROCESS /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/noapoe/af0.01/model1_skat/Hispanic.pca.projected_outlier_removed_AD.SMMAT.snp_stats.gz

OUTFILE /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/noapoe/af0.01/model1_skat/ADSP.SMMAT_noAPOE4_META .TXT
ANALYZE
ANALYZE HETEROGENEITY

In [10]:
# Reformat metal results to map chromosome and position
library(stringr) # to replace strings
library(tidyr) # to get  separate function separate the marker name
library(data.table)
library(dplyr)

In [3]:
data = read.table('/mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/noapoe/af0.01/model1_skat/ADSP.SMMAT_noAPOE4_META2.TXT', header = T, sep = '\t')
his = read.table('/mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/noapoe/af0.01/model1_skat/Hispanic.pca.projected_outlier_removed_AD.SMMAT.snp_stats.gz', header = T, sep ='\t')[, c(1, 2, 3)]
eur = read.table('/mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/noapoe/af0.01/model1_skat/European.pca.projected_outlier_removed_AD.SMMAT.snp_stats.gz', header = T, sep ='\t')[, c(1, 2, 3)]
afr = read.table('/mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/noapoe/af0.01/model1_skat/African.pca.projected_outlier_removed_AD.SMMAT.snp_stats.gz', header = T, sep ='\t')[, c(1, 2, 3)]
pos = merge(his, eur, by = c('CHR', 'POS', 'SNP'),all = TRUE)
POS = merge(pos, afr, by = c('CHR', 'POS', 'SNP'),all = TRUE)
data = inner_join(data, POS, by = c('MarkerName'='SNP'))
data$POS = as.numeric(data$POS)
data$CHR = as.numeric(data$CHR)
write.table(data,'/mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/noapoe/af0.01/model1_skat/meta.smmat_noAPOE4.txt', sep = '\t', quote = F, col.names = T, row.names = F)
lambda <- median(qchisq(1-data$P.value,1), na.rm=TRUE)/qchisq(0.5,1)

### Model 2 without adjusting for APOE4

In [ ]:
SCHEME SAMPLESIZE
MARKER  SNP
WEIGHT  N
EFFECT  BETA
PVAL    P


PROCESS /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/noapoe/af0.01/model3_skat/African.pca.projected_outlier_removed_AD.SMMAT.snp_stats.gz
PROCESS /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/noapoe/af0.01/model3_skat/European.pca.projected_outlier_removed_AD.SMMAT.snp_stats.gz
PROCESS /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/noapoe/af0.01/model3_skat/Hispanic.pca.projected_outlier_removed_AD.SMMAT.snp_stats.gz

OUTFILE /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/noapoe/af0.01/model3_skat/ADSP.SMMAT_noAPOE4_META .TXT
ANALYZE
ANALYZE HETEROGENEITY

In [5]:
data = read.table('/mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/noapoe/af0.01/model3_skat/ADSP.SMMAT_noAPOE4_META2.TXT', header = T, sep = '\t')
his = read.table('/mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/noapoe/af0.01/model3_skat/Hispanic.pca.projected_outlier_removed_AD.SMMAT.snp_stats.gz', header = T, sep ='\t')[, c(1, 2, 3)]
eur = read.table('/mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/noapoe/af0.01/model3_skat/European.pca.projected_outlier_removed_AD.SMMAT.snp_stats.gz', header = T, sep ='\t')[, c(1, 2, 3)]
afr = read.table('/mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/noapoe/af0.01/model3_skat/African.pca.projected_outlier_removed_AD.SMMAT.snp_stats.gz', header = T, sep ='\t')[, c(1, 2, 3)]
pos = merge(his, eur, by = c('CHR', 'POS', 'SNP'),all = TRUE)
POS = merge(pos, afr, by = c('CHR', 'POS', 'SNP'),all = TRUE)
data = inner_join(data, POS, by = c('MarkerName'='SNP'))
data$POS = as.numeric(data$POS)
data$CHR = as.numeric(data$CHR)
write.table(data,'/mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/noapoe/af0.01/model3_skat/meta.smmat_noAPOE4.txt', sep = '\t', quote = F, col.names = T, row.names = F)
lambda <- median(qchisq(1-data$P.value,1), na.rm=TRUE)/qchisq(0.5,1)

### Model 1 with adjusting for APOE4

In [ ]:
SCHEME SAMPLESIZE
MARKER  SNP
WEIGHT  N
EFFECT  BETA
PVAL    P


PROCESS /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/apoe/af0.01/model1_skat/African.pca.projected_outlier_removed_AD.SMMAT.snp_stats.gz
PROCESS /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/apoe/af0.01/model1_skat/European.pca.projected_outlier_removed_AD.SMMAT.snp_stats.gz
PROCESS /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/apoe/af0.01/model1_skat/Hispanic.pca.projected_outlier_removed_AD.SMMAT.snp_stats.gz

OUTFILE /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/apoe/af0.01/model1_skat/ADSP.SMMAT_APOE4_META .TXT
ANALYZE
ANALYZE HETEROGENEITY

In [8]:
data = read.table('/mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/apoe/af0.01/model1_skat/ADSP.SMMAT_APOE4_META2.TXT', header = T, sep = '\t')
his = read.table('/mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/apoe/af0.01/model1_skat/Hispanic.pca.projected_outlier_removed_AD.SMMAT.snp_stats.gz', header = T, sep ='\t')[, c(1, 2, 3)]
eur = read.table('/mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/apoe/af0.01/model1_skat/European.pca.projected_outlier_removed_AD.SMMAT.snp_stats.gz', header = T, sep ='\t')[, c(1, 2, 3)]
afr = read.table('/mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/apoe/af0.01/model1_skat/African.pca.projected_outlier_removed_AD.SMMAT.snp_stats.gz', header = T, sep ='\t')[, c(1, 2, 3)]
pos = merge(his, eur, by = c('CHR', 'POS', 'SNP'),all = TRUE)
POS = merge(pos, afr, by = c('CHR', 'POS', 'SNP'),all = TRUE)
data = inner_join(data, POS, by = c('MarkerName'='SNP'))
data$POS = as.numeric(data$POS)
data$CHR = as.numeric(data$CHR)
write.table(data,'/mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/apoe/af0.01/model1_skat/meta.smmat_APOE4.txt', sep = '\t', quote = F, col.names = T, row.names = F)
lambda <- median(qchisq(1-data$P.value,1), na.rm=TRUE)/qchisq(0.5,1)

### Model 2 with adjusting for APOE4

In [ ]:
SCHEME SAMPLESIZE
MARKER  SNP
WEIGHT  N
EFFECT  BETA
PVAL    P


PROCESS /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/apoe/af0.01/model3_skat/African.pca.projected_outlier_removed_AD.SMMAT.snp_stats.gz
PROCESS /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/apoe/af0.01/model3_skat/European.pca.projected_outlier_removed_AD.SMMAT.snp_stats.gz
PROCESS /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/apoe/af0.01/model3_skat/Hispanic.pca.projected_outlier_removed_AD.SMMAT.snp_stats.gz

OUTFILE /mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/apoe/af0.01/model3_skat/ADSP.SMMAT_APOE4_META .TXT
ANALYZE
ANALYZE HETEROGENEITY

In [9]:
data = read.table('/mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/apoe/af0.01/model3_skat/ADSP.SMMAT_APOE4_META2.TXT', header = T, sep = '\t')
his = read.table('/mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/apoe/af0.01/model3_skat/Hispanic.pca.projected_outlier_removed_AD.SMMAT.snp_stats.gz', header = T, sep ='\t')[, c(1, 2, 3)]
eur = read.table('/mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/apoe/af0.01/model3_skat/European.pca.projected_outlier_removed_AD.SMMAT.snp_stats.gz', header = T, sep ='\t')[, c(1, 2, 3)]
afr = read.table('/mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/apoe/af0.01/model3_skat/African.pca.projected_outlier_removed_AD.SMMAT.snp_stats.gz', header = T, sep ='\t')[, c(1, 2, 3)]
pos = merge(his, eur, by = c('CHR', 'POS', 'SNP'),all = TRUE)
POS = merge(pos, afr, by = c('CHR', 'POS', 'SNP'),all = TRUE)
data = inner_join(data, POS, by = c('MarkerName'='SNP'))
data$POS = as.numeric(data$POS)
data$CHR = as.numeric(data$CHR)
write.table(data,'/mnt/mfs/statgen/alzheimers-family/ADSP_2023/smmat/WES/apoe/af0.01/model3_skat/meta.smmat_APOE4.txt', sep = '\t', quote = F, col.names = T, row.names = F)
lambda <- median(qchisq(1-data$P.value,1), na.rm=TRUE)/qchisq(0.5,1)